 <font size = 1000px color = 'royalblue'><center><b>Blending<b></center></font>
 

<font color = "#191970" size=6px><center><b>Import Required Libraries</b></font>

In [ ]:
# Data Wrangling and Data Analysis

import pandas as pd , numpy as np
import gc

# Visualization

from matplotlib import pyplot as plt, style
import seaborn as sns

# Feature Engineering / Feature Selection
import optuna 
from optuna.integration import XGBoostPruningCallback
from sklearn.preprocessing import RobustScaler
from sklearn.model_selection import train_test_split


# Model Building
#import lightgbm as lgbm
#from lightgbm import LGBMClassifier
from xgboost import XGBClassifier
#from catboost import CatBoostClassifier, Pool
from sklearn import metrics


# Ignore Warings

import warnings
warnings.filterwarnings("ignore")

<font color = "#191970" size=6px><center><b>Data Collection</b></font>

<font color = "maroon" size=5px><b>1.1 Reducing Data Size</b></font>

<font color = "maroon" size=4px><b>Since the dataset is vert large(1 million rows) . lest use below function to convert each column to the best fitting datatype for its range.</b></font>

In [ ]:
def reduce_mem_usage(df):
    """ iterate through all the columns of a dataframe and modify the data type
        to reduce memory usage.        
    """
    start_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage of dataframe is {:.2f} MB'.format(start_mem))
    
    for col in df.columns:
        col_type = df[col].dtype
        
        if col_type != object:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
        else:
            df[col] = df[col].astype('category')

    end_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage after optimization is: {:.2f} MB'.format(end_mem))
    print('Decreased by {:.1f}%'.format(100 * (start_mem - end_mem) / start_mem))
    
    return df

In [ ]:
# Before working with the data, we reduce the use of memory, so we can improve performance

def import_data(file):
    # Reading File
    df = pd.read_csv(file)
    
    # Reducing Size by Optimizing Dtypes of columns
    df = reduce_mem_usage(df)
    
    # Converting Bool cols into integer
    bool_cols = []
    for i, col in enumerate(df.columns):
        if df[col].dtypes == bool:
            bool_cols.append(i)
    df.iloc[:, bool_cols] = df.iloc[:, bool_cols].astype(int)  
    
    return df

# Train Data
train = import_data('../input/tpsoct2021-5-folds/Train_tps_oct_2021_kfold.csv')

#Test Data
test = import_data('../input/tabular-playground-series-oct-2021/test.csv')


# Submission

submission = import_data('../input/tabular-playground-series-oct-2021/sample_submission.csv')

<font color = "maroon" size=5px><b>2.3 Analysing Different Column Types</b></font>

In [ ]:
# categorical Columns :- 

Cat_cols = [col for col in train.columns if train[col].nunique() < 5]
Cat_cols.remove('target')
#Cat_cols_indices = [train.columns.get_loc(col) for col in Cat_cols]

# Continous Columns : -

Con_cols = [col for col in train.columns if train[col].nunique() > 5]



# Number of each respective column types

print("Number of Categorical Columns :", len(Cat_cols))
print("Number of Continous Columns   :", len(Con_cols))

gc.collect()

<font color = "blue" size=4px> <b>Interpretation : </b></font>

* <font color = "red" size=4px>Most features Are Left Skewed, so We use Algorithms That Doesnt Expect Normality ,i.e any **tree based ensemble techniques** </font>

<hr>

<font color = "maroon" size=5px><b>2.4 Feature Engineering</b></font>

In [ ]:
Con_cols_test = [cols for cols in Con_cols if cols not in ["mean", "std", "min", "max", "abs_sum", "sem"]]
test["mean"] = test[Con_cols_test].mean(axis=1)
test["std"] = test[Con_cols_test].std(axis=1)
test["min"] = test[Con_cols_test].min(axis=1)
test["max"] = test[Con_cols_test].max(axis=1)
test['abs_sum'] = test[Con_cols_test].abs().sum(axis=1)
test['sem'] = test[Con_cols_test].sem(axis=1)
del Con_cols_test
gc.collect()

<font color = "#191970" size=6px><center><b>XGB Model</b></font>

<font color = "maroon" size=5px><b>3.1 Best Params From HyperParameter Tuning Notebook :- [HyperParameter Tuning TPS OCT-2021](https://www.kaggle.com/jaysrivastav/blending-tps-oct-21?scriptVersionId=77672359) </b></font>

In [ ]:
XGB_params = {'learning_rate': 0.014679233453195013, 'reg_lamba':75.56651890088857, 'reg_alpha': 0.11766857055687065,
              'gamma': 0.6407823221122686, 'subsample': 0.4640789338167099,
              'subsample': 0.7,'colsample_bytree': 0.2,'colsample_bylevel': 0.6000000000000001,
              'min_child_weight': 56.41980735551558,'max_depth': 6,
              'verbosity' : 0, 'eval_metric' : 'auc','objective':"binary:logistic",
              'use_label_encoder': False,'tree_method': 'gpu_hist',
              "seed": 42, 'n_jobs': -1, 'n_estimators': 20000, 'predictor': 'gpu_predictor',
              'learning_rate': 0.013474548048574765}


In [ ]:
# Features and Target 

train.reset_index(inplace=True)
test.reset_index(inplace=True)
Features = [c for c in train.columns if c not in ('id','index','target','kfold')]
gc.collect()

In [ ]:
# Kfold Loop

def XGB():
    final_test_predictions = []
    final_valid_predictions = {}
    scores = []
    for fold in range(5):
    
        x_train = train[train.kfold != fold].reset_index(drop=True)
        x_valid = train[train.kfold == fold].reset_index(drop=True)
        x_test = test[Features].copy()
    
        valid_ids = x_valid.index.values.tolist()
   
        y_train = x_train.target
        y_valid = x_valid.target
    
        x_train = x_train[Features]
        x_valid = x_valid[Features]
    
   
    
        gc.collect()

        # Model Training
        model = XGBClassifier(**XGB_params, n_threads=4)
        model.fit(x_train, y_train,
                  eval_set=[(x_valid,y_valid)],
                  early_stopping_rounds=200,
                  verbose = False)
        gc.collect()
    
        # Predictions
        valid_preds = model.predict_proba(x_valid)[:,1]
        test_preds = model.predict_proba(x_test)[:,1]
    
        final_test_predictions.append(test_preds)
        final_valid_predictions.update(dict(zip(valid_ids, valid_preds)))

        # Score
        auc = metrics.roc_auc_score(y_valid, valid_preds)
        scores.append(auc)
        print(f"Fold: {fold + 1} Score: {auc}")
        print('||'*40)
        gc.collect()
    print("Overall Auc : - ", np.mean(scores))
    return final_test_predictions, final_valid_predictions
    
final_test_predictions, final_valid_predictions = XGB()

In [ ]:
# final_valid_predictions = pd.DataFrame.from_dict(final_valid_predictions, orient = "index")
# final_valid_predictions.columns = ['id', 'pred_1']

# final_valid_predictions.to_csv("train_pred_1.csv", index = False)

In [ ]:
submission.target = np.mean(np.column_stack(final_test_predictions), axis = 1)

#submission.columns = ['id', 'pred_1']
submission.to_csv("submission.csv", index = False)

In [ ]:
submission.head()

<font color = "#191970" size=6px><center><b>LGBM HyperParameter Tuning</b></font>

In [ ]:
# lgb_params = {
#      'objective': 'binary',
#      'n_estimators':N_ESTIMATORS,
#      'importance_type': 'gain',
#      'metric':'auc',
#      'boosting_type': 'gbdt',
#      'n_jobs' : -1, 
#     'learning_rate': 0.0038511441056118664, 
#     'subsample': 0.5827550088149794, 
#     'subsample_freq': 1, 
#     'colsample_bytree': 0.19599597755538956, 
#     'reg_lambda': 0.011685550612519125, 
#     'reg_alpha': 0.04502045156737212, 
#     'min_child_weight': 16.843316711276092, 
#     'min_child_samples': 412, 
#     'num_leaves': 546, 
#     'max_depth': 5, 
#     'cat_smooth': 36.40200359200525, 
#     'cat_l2': 12.979520035205597
#     }